### Ввод-вывод средствами С и C++

**Задача**: посчитать сумму 500 000 целых чисел из потока ввода.

Решим эту задачу средствами С и С++ и оценим различия в подходах.

Сгенерируем тестовые данные:

In [1]:
with open('input.txt', 'w') as f:
    f.write(' '.join(['1'] * 500000))

Решение задачи с использованием средства ввода-вывода С:

In [ ]:
# %load inout/main_c.cpp
#include <cstdio>


int main()
{
	int sum = 0;
	for (int i = 0; i < 500'000; ++i)
	{
		int x = 0;
		std::scanf("%i", &x);
		sum += x;
	}
	std::printf("%i\n", sum);
	return 0;
}



Со спецификацией форматов можно ознакомиться [здесь](https://en.cppreference.com/w/cpp/io/c/fprintf)

Решение задачи с использованием средства ввода-вывода С++:

In [ ]:
# %load inout/main_cpp.cpp
#include <iostream>


int main()
{
	int sum = 0;
	for (int i = 0; i < 500'000; ++i)
	{
		int x = 0;
		std::cin >> x;
		sum += x;
	}
	std::cout << sum << std::endl;
	return 0;
}



Скомпилируем решения:

In [2]:
!clang++ --version

clang version 6.0.0-1ubuntu2 (tags/RELEASE_600/final)
Target: x86_64-pc-linux-gnu
Thread model: posix
InstalledDir: /usr/bin


In [3]:
!time -f "execution time: %E" clang++ -O3 inout/main_c.cpp -o inout_c.exe

execution time: 0:00.08


In [4]:
!time -f "execution time: %E" clang++ -O3 inout/main_cpp.cpp -o inout_cpp.exe

execution time: 0:00.28


Проверим размеры исполняемых файлов

In [5]:
!ls -l *.exe

-rwxrwxr-x 1 ivafanas ivafanas 8240 июл 29 22:52 inout_c.exe
-rwxrwxr-x 1 ivafanas ivafanas 8880 июл 29 22:52 inout_cpp.exe


Зайдём на godbolt.org и посмотрим на причину, почему исполняемый файл для С++ - решения стал шире

Запустим, проверим, какое работает быстрее

In [6]:
!time -f "execution time: %E" cat input.txt | ./inout_c.exe

execution time: 0:00.02
500000


In [7]:
!time -f "execution time: %E" cat input.txt | ./inout_cpp.exe

execution time: 0:00.05
500000


Вопросы для обсуждения:
* Почему?
* Всегда ли решение в стиле С работает быстрее?

<br />

Сравнение:

С:
* быстрая компиляция
* быстрое выполнение
* меньше размер исходного файла

C++:
* типобезопасно

  ```
  scanf(**"%i"**, &i);
  ```


* меньше ошибок при множественной записи / чтении:
  ```
  printf("%i %f %u", i, z, n);
  std::cout << i << ' ' << z << ' ' << n;
  ```

* меньше ошибок с адресной арифметикой

  ```
  scanf("%i", **&**i);
  ```


<br />

**Вывод**: если программа упирается в ввод-вывод - используем С - вариант, если нет - С++ - вариант.

<br />

In [8]:
# cleanup
!rm -f *.exe input.txt

<br />

### Работа с командной строкой

Общепринят формат передачи аргументов командной строки в программу, в котором аргументы делятся на:
* позиционные
* флажки
* именованные со значением

Пример программы `ls`, выводящей содержимое папки:

```
ls /usr/bin -a --human-readable --color=always
ls -a --color always --human-readable /usr/bin
```

описание опций можно прочитать, набрав `ls --help`:

```
  -a, --all                  do not ignore entries starting with .
  -h, --human-readable       with -l and/or -s, print human readable sizes
                               (e.g., 1K 234M 2G)
      --color[=WHEN]         colorize the output; WHEN can be 'always' (default
                               if omitted), 'auto', or 'never'; more info below
```

**Задача**: вывести аргументы командной строки, переданные консольной утилите

In [ ]:
# %load cmdline/main.cpp
#include <iostream>


int main(int argc, char** argv)
{
	std::cout << "command line arguments are:" << std::endl;
	for (int i = 0; i < argc; ++i)
	{
		std::cout << "    " << i << " -> " << argv[i] << std::endl;
	}
	return 0;
}



Скомпилируем

In [9]:
!clang++ -O3 cmdline/main.cpp -o a.out

Протестируем

In [10]:
!./a.out /usr/bin -a --human-readable --color=always

command line arguments are:
    0 -> ./a.out
    1 -> /usr/bin
    2 -> -a
    3 -> --human-readable
    4 -> --color=always


In [11]:
!./a.out /usr/bin -a --human-readable --color always

command line arguments are:
    0 -> ./a.out
    1 -> /usr/bin
    2 -> -a
    3 -> --human-readable
    4 -> --color
    5 -> always


In [12]:
!./a.out "/usr/bin/my awesome app dir" -a --human-readable --color=always

command line arguments are:
    0 -> ./a.out
    1 -> /usr/bin/my awesome app dir
    2 -> -a
    3 -> --human-readable
    4 -> --color=always


Приберём мусор

In [13]:
# cleanup
!rm -f a.out

<br />

Готовые решения:

* C++, cross-platform [`boost::program_options`](https://www.boost.org/doc/libs/1_58_0/doc/html/program_options.html)
* C, unix-only [`getopt`](https://www.gnu.org/software/libc/manual/html_node/Example-of-Getopt.html)

<br />

### Время компиляции

**Задача:** посчитать приближение `n`-ой степени числа `e` по формуле `(1 + 1/n)^n`, где `n` вводится пользователем с клавиатуры.

Решение данной задачи тривиально, а мы рассмотрим две возможные реализации решения.

Вариант 1:

In [ ]:
# %load exp/ex1/util.h
#pragma once

#include <iostream>


unsigned read_n(std::istream& is);

void write_result(double res, std::ostream& os);

double calc_exp(unsigned n);



In [ ]:
# %load exp/ex1/util.cpp
#include "util.h"

#include <cmath>

unsigned read_n(std::istream& is)
{
	unsigned rv = 0;
	std::cin >> rv;
	return rv;
}

void write_result(double res, std::ostream& os)
{
	os << res << std::endl;
}

double calc_exp(unsigned n)
{
	double b = 1 + 1.0 / n;
	return std::pow(b, n);
}



In [ ]:
# %load exp/ex1/solver.h
#pragma once

#include <iostream>

void solve(std::istream& is, std::ostream& os);


In [ ]:
# %load exp/ex1/solver.cpp
#include "util.h"


void solve(std::istream& is, std::ostream& os)
{
	const unsigned n = read_n(is);
	const double res = calc_exp(n);
	write_result(res, os);
}



In [ ]:
# %load exp/ex1/main.cpp
#include "solver.h"

#include <iostream>


int main()
{
	solve(std::cin, std::cout);
	return 0;
}


Вариант 2:

In [ ]:
# %load exp/ex2/util.h
#pragma once

#include <iosfwd>


unsigned read_n(std::istream& is);

void write_result(double res, std::ostream& os);

double calc_exp(unsigned n);



In [ ]:
# %load exp/ex2/util.cpp
#include "util.h"

#include <cmath>
#include <iostream>


unsigned read_n(std::istream& is)
{
	unsigned rv = 0;
	is >> rv;
	return rv;
}

void write_result(double res, std::ostream& os)
{
	os << res << std::endl;
}

double calc_exp(unsigned n)
{
	double b = 1 + 1.0 / n;
	return std::pow(b, n);
}



In [ ]:
# %load exp/ex2/solver.h
#pragma once

#include <iosfwd>

void solve(std::istream& is, std::ostream& os);


In [ ]:
# %load exp/ex2/solver.cpp
#include "util.h"


void solve(std::istream& is, std::ostream& os)
{
	const unsigned n = read_n(is);
	const double res = calc_exp(n);
	write_result(res, os);
}



In [ ]:
# %load exp/ex2/main.cpp
#include "solver.h"

#include <iostream>


int main()
{
	solve(std::cin, std::cout);
	return 0;
}


Измерим время компиляции:

In [14]:
!time -f "execution time: %E" clang++ -O3 exp/ex1/main.cpp exp/ex1/solver.cpp exp/ex1/util.cpp -o exponent_1.exe

execution time: 0:00.77


In [15]:
!time -f "execution time: %E" clang++ -O3 exp/ex2/main.cpp exp/ex2/solver.cpp exp/ex2/util.cpp -o exponent_2.exe

execution time: 0:00.57


Проверим работоспособность:

In [16]:
!echo 100 | ./exponent_1.exe
!echo 100 | ./exponent_2.exe

2.70481
2.70481


Почему разница во времени компиляции 200 мс?

In [17]:
# cleanup
!rm -f *.exe